# Imports

## Libraries & Packages

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import nltk

## Data

In [100]:
PATH = "../Data/"

In [102]:
df = pd.read_csv(PATH + "CASE & SURVEY.csv")

In [159]:
drop_cols = ["DURATION_HRS", "RESOLVER_USERNAME",
             "RESOLVER_NAME", "CASE_OWNER_S_MANAGER_EMAIL__C"
            ]

df.drop(columns=drop_cols, inplace=True)

In [161]:
df

,CASE_NUMBER,CASE_SYMPTOM,CASE_DIAGNOSIS,CASE_STATUS,CASE_ORIGIN,CASE_CREATED_DATETIME,CASE_CLOSED_DATETIME,ISESCALATED,GROUPING,EFFORT_RATING__C,RATING__C,ADDITIONAL_COMMENTS__C,NPS_POSITIVE,NPS_NEUTRAL,NPS_NEGATIVE,DURATION_INSECONDS,DURATION_INMINUTES
0,13673869,Application Issue,Application How to Use,Closed,Contact Us,2024-06-24 08:48:04,2024-06-25 06:41:54,False,TEAM 1,Strongly Agree,10.0,NaN,1,0,0,78830.0,1313.833333
1,13668140,Application Issue,Application Malfunction,Closed,Phone,2024-06-20 20:04:30,2024-06-20 22:33:02,False,TEAM 1,Strongly Agree,10.0,NaN,1,0,0,8912.0,148.533333
2,13674782,Login Failure,Application Malfunction,Closed,Phone,2024-06-24 12:29:30,2024-06-24 18:41:58,False,TEAM 3,Agree,9.0,NaN,1,0,0,22348.0,372.466667
3,13669283,How To Use,Content Explanation,Closed,Contact Us,2024-06-21 07:30:45,2024-06-24 05:48:06,False,TEAM 3,Somewhat Agree,6.0,NaN,0,1,0,253041.0,4217.350000
4,13674563,Information Request,Application Malfunction,Closed,Phone,2024-06-24 11:43:59,2024-06-24 16:59:58,False,TEAM 1,Agree,10.0,NaN,1,0,0,18959.0,315.983333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4521,13225577,Connectivity,Application Malfunction,Closed,Chat,2024-01-16 08:24:38,2024-01-16 09:46:02,False,TEAM 2,Agree,8.0,could be faster,1,0,0,4884.0,81.400000
4522,13220950,Content Explanation,Content Explanation,Closed,Chat,2024-01-14 11:37:11,2024-01-15 08:20:59,False,TEAM 1,Strongly Agree,10.0,Thank you very much.,1,0,0,74628.0,1243.800000
4523,13207286,Content Explanation,Content Explanation,Closed,Email,2024-01-09 10:29:57,2024-01-12 14:23:19,False,TEAM 1,Strongly Disagree,0.0,It took 2 weeks for my query to be answered an...,0,0,1,273202.0,4553.366667
4524,13203866,Application Issue,Application Malfunction,Closed,Contact Us,2024-01-08 12:50:31,2024-01-12 22:01:21,False,TEAM 3,Strongly Agree,10.0,Paul Massoni and Chris Matheis are the best.,1,0,0,378650.0,6310.833333


# EDA

In [104]:
df.head()

,CASE_NUMBER,CASE_SYMPTOM,CASE_DIAGNOSIS,CASE_STATUS,CASE_ORIGIN,CASE_CREATED_DATETIME,CASE_CLOSED_DATETIME,DURATION_HRS,RESOLVER_USERNAME,RESOLVER_NAME,CASE_OWNER_S_MANAGER_EMAIL__C,ISESCALATED,GROUPING,EFFORT_RATING__C,RATING__C,ADDITIONAL_COMMENTS__C,NPS_POSITIVE,NPS_NEUTRAL,NPS_NEGATIVE
0,13673869,Application Issue,Application How to Use,Closed,Contact Us,2024-06-24T08:48:04.000Z,2024-06-25T06:41:54.000Z,21.897222,marrionclarisse.descallar@lseg.com,Marrion Clarisse Descallar,leo.limpangog@refinitiv.com,False,TEAM 1,Strongly Agree,10.0,NaN,1,0,0
1,13668140,Application Issue,Application Malfunction,Closed,Phone,2024-06-20T20:04:30.000Z,2024-06-20T22:33:02.000Z,2.475556,deck.salvador@refinitiv.com,Decoloroso Salvador,emmanuel.elopre@refinitiv.com,False,TEAM 1,Strongly Agree,10.0,NaN,1,0,0
2,13674782,Login Failure,Application Malfunction,Closed,Phone,2024-06-24T12:29:30.000Z,2024-06-24T18:41:58.000Z,6.207778,marvin.joson@lseg.com,Marvin Joson,ross.lindaya@refinitiv.com,False,TEAM 3,Agree,9.0,NaN,1,0,0
3,13669283,How To Use,Content Explanation,Closed,Contact Us,2024-06-21T07:30:45.000Z,2024-06-24T05:48:06.000Z,70.289167,mayangelu.beloso@lseg.com,May Angelu Beloso,sheila.navarro@refinitiv.com,False,TEAM 3,Somewhat Agree,6.0,NaN,0,1,0
4,13674563,Information Request,Application Malfunction,Closed,Phone,2024-06-24T11:43:59.000Z,2024-06-24T16:59:58.000Z,5.266389,jeffrey.par@refinitiv.com,Jeffrey Par,emmanuel.elopre@refinitiv.com,False,TEAM 1,Agree,10.0,NaN,1,0,0


In [106]:
df["CASE_SYMPTOM"].value_counts()

CASE_SYMPTOM
Application Issue      1656
How To Use              774
Content Explanation     359
Login Failure           357
Content Not Found       317
Information Request     272
Content Integrity       238
Password Assistance     219
Add-in Missing          214
Connectivity            102
Content Addition          8
Content Slowness          6
Technical                 4
Name: count, dtype: int64

In [108]:
df["CASE_DIAGNOSIS"].value_counts()

CASE_DIAGNOSIS
Application/Desktop/Malfunction             792
Content Explanation                         567
Application Malfunction                     505
Content Search                              447
Application/Desktop/How to Use              318
Application How to Use                      278
Application/Desktop/Crash                   232
Manual Password Reset                       169
Password Reset                              112
Application/Desktop/Func. NA                107
Application/Desktop/Install                  75
Application Crash                            75
Content Not Available                        71
Entitlement Confirm ID/Perm                  62
Application Func. Not Available              62
Application/Infrastructure/How to Use        57
Application/Desktop/File Conversion          55
Application/Desktop/How to Use               54
Application Install                          47
Self-password Reset Issue                    46
Content Incorrect        

In [110]:
df["CASE_STATUS"].value_counts()

CASE_STATUS
Closed       4501
Cancelled      23
Rejected        1
Delivered       1
Name: count, dtype: int64

# Feature Engineering

### Convert Timezone

In [113]:
df["CASE_CREATED_DATETIME"] = pd.to_datetime(df["CASE_CREATED_DATETIME"], utc=True).dt.tz_localize(None)
df["CASE_CLOSED_DATETIME"] = pd.to_datetime(df["CASE_CLOSED_DATETIME"], utc=True).dt.tz_localize(None)

### Get Duration
- Total Seconds
- Total Minutes

In [142]:
df["DURATION_INSECONDS"] = (df["CASE_CLOSED_DATETIME"] - df["CASE_CREATED_DATETIME"]) / pd.Timedelta(seconds=1)
df["DURATION_INMINUTES"] = (df["CASE_CLOSED_DATETIME"] - df["CASE_CREATED_DATETIME"]) / pd.Timedelta(minutes=1)

0      0 days 21:53:50
1      0 days 02:28:32
2      0 days 06:12:28
3      2 days 22:17:21
4      0 days 05:15:59
             ...      
4521   0 days 01:21:24
4522   0 days 20:43:48
4523   3 days 03:53:22
4524   4 days 09:10:50
4525   0 days 01:41:27
Length: 4526, dtype: timedelta64[ns]

### Regex Cleanup

In [64]:
df[df["CASE_NUMBER"] == 13671561]

,CASE_NUMBER,CASE_SYMPTOM,CASE_DIAGNOSIS,CASE_STATUS,CASE_ORIGIN,CASE_CREATED_DATETIME,CASE_CLOSED_DATETIME,DURATION_HRS,RESOLVER_USERNAME,RESOLVER_NAME,CASE_OWNER_S_MANAGER_EMAIL__C,ISESCALATED,GROUPING,EFFORT_RATING__C,RATING__C,ADDITIONAL_COMMENTS__C,NPS_POSITIVE,NPS_NEUTRAL,NPS_NEGATIVE
19,13671561,Information Request,Content Not Available,Closed,Chat,2024-06-21 19:37:12,2024-06-24 22:14:50,74.627222,jayarama.-@refinitiv.com,Jayarama -,celine.gonzales@refinitiv.com,False,TEAM 2,Agree,9.0,.,1,0,0
